# Run Correlation-Based Analyses

### Authors: Calvin Howard.

#### Last updated: July 6, 2023

Use this to assess if a correlation between a dependent variable and an independent variable is statistically significant using permutation analysis. 

Further, follow this up with a contrast analysis which sees which categorical variables have significantly different correlations from each other. 

Notes:
- To best use this notebook, you should be familar with mixed effects models

# 00 - Import CSV with All Data
**The CSV is expected to be in this format**
- ID and absolute paths to niftis are critical
```
+-----+----------------------------+--------------+--------------+--------------+
| ID  | Nifti_File_Path            | Covariate_1  | Covariate_2  | Covariate_3  |
+-----+----------------------------+--------------+--------------+--------------+
| 1   | /path/to/file1.nii.gz      | 0.5          | 1.2          | 3.4          |
| 2   | /path/to/file2.nii.gz      | 0.7          | 1.4          | 3.1          |
| 3   | /path/to/file3.nii.gz      | 0.6          | 1.5          | 3.5          |
| 4   | /path/to/file4.nii.gz      | 0.9          | 1.1          | 3.2          |
| ... | ...                        | ...          | ...          | ...          |
+-----+----------------------------+--------------+--------------+--------------+
```

In [ ]:
# Specify the path to your CSV file containing NIFTI paths
input_csv_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/resources/datasets/TMS-DBS_studies_alzheimers/metadata/secondary_list.csv'
sheet = None #'master_list'

In [ ]:
# Specify where you want to save your results to
out_dir = None

In [ ]:
from calvin_utils.permutation_analysis_utils.statsmodels_palm import CalvinStatsmodelsPalm
# Instantiate the PalmPrepararation class
cal_palm = CalvinStatsmodelsPalm(input_csv_path=input_csv_path, output_dir=out_dir, sheet=sheet)
# Call the process_nifti_paths method
data_df = cal_palm.read_and_display_data()
data_df


# 01 - Preprocess Your Data

**Handle NANs**
- Set drop_nans=True is you would like to remove NaNs from data
- Provide a column name or a list of column names to remove NaNs from

In [ ]:
data_df.columns

In [ ]:
drop_list = ['Overall_Cognition_Cohen_s_D', 'all_flipped_all_datasets_weighted_r_map_pcc_inverse_r_map_average_target_in_subject']

In [ ]:
data_df = cal_palm.drop_nans_from_columns(columns_to_drop_from=drop_list)
data_df

**Drop Row Based on Value of Column**

Define the column, condition, and value for dropping rows
- column = 'your_column_name'
- condition = 'above'  # Options: 'equal', 'above', 'below'

Set the parameters for dropping rows

In [ ]:
data_df.columns

In [ ]:
column = 'Disease'  # The column you'd like to evaluate
condition = 'not'  # Thecondition to check ('equal', 'above', 'below', 'not')
value = 'Parkinson' # The value to compare against

In [ ]:
data_df, other_df = cal_palm.drop_rows_based_on_value(column, condition, value)
data_df

Regress out a Covariate

In [ ]:
for col in data_df.columns:
    
    print(col)

In [ ]:
dependent_variable_list = ['Pre_Post_Memory_Effect_Size__Cohen_s_D_']
regressors = ['Disease', 'Years_Between_Measurements']

In [ ]:
for col in regressors:
    if data_df[col].dtype == 'O':  # object/categorical
        data_df[col] = data_df[col].fillna(data_df[col].mode()[0])
    else:  # numeric
        data_df[col] = data_df[col].fillna(data_df[col].mean())
data_df[regressors].isna().sum()


In [ ]:
from calvin_utils.statistical_utils.regression_utils import RegressOutCovariates
data_df, adjusted_dep_vars_list = RegressOutCovariates.run(df=data_df, dependent_variable_list=dependent_variable_list, covariates_list=regressors)
print(adjusted_dep_vars_list)

In [ ]:
import numpy as np
nonzero_count = np.count_nonzero(data_df['Pre_Post_Memory_Effect_Size__Cohen_s_D__residual'])
not_nan_count = data_df['Pre_Post_Memory_Effect_Size__Cohen_s_D__residual'].notna().sum()
print(f"Nonzero count: {nonzero_count}")
print(f"Non-NaN count: {not_nan_count}")

**Standardize Data**
- Enter Columns you Don't want to standardize into a list

In [ ]:
data_df.columns

In [ ]:
# Remove anything you don't want to standardize
cols_not_to_standardize = ['TOTAL11']

In [ ]:
# data_df = cal_palm.standardize_columns(cols_not_to_standardize)
data_df

Misc Filtering

In [ ]:
# data_df = data_df[data_df['Pre_Post_Memory_Effect_Size__Cohen_s_D_'].abs() <= 10]

# 02 - Perform Basic Correlation

In [ ]:

data_df.columns

In [ ]:
x_col = 'all_flipped_all_datasets_weighted_r_map_pcc_inverse_r_map_average_target_in_subject'
y_col = 'Overall_Cognition_Cohen_s_D'

In [ ]:
out_dir = '/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Academia/Research/2023/ccm_memory/Figures/explaining_prior_targets_alzheimer'

In [ ]:
from calvin_utils.statistical_utils.scatterplot import simple_scatter
simple_scatter(data_df, x_col, y_col, 'DBS', 
               x_label="Spatial Correlation",
               y_label="Cognitive Outcome (Cohen's D)",
               out_dir=out_dir, flip_axes=False)

In [ ]:
from scipy.stats import ttest_ind
from scipy.stats import mannwhitneyu
from scipy.stats import wilcoxon

# Split data into two groups based on x_col positive vs negative
group_pos = data_df[data_df[x_col] > 0][y_col].dropna()
group_neg = data_df[data_df[x_col] < 0][y_col].dropna()

# Perform independent t-test
t_stat, p_val = ttest_ind(group_pos, group_neg, equal_var=False)

print(f"T-test: T-statistic = {t_stat:.4f}, p-value = {p_val:.4g}")
print(f"Group sizes: positive={len(group_pos)}, negative={len(group_neg)}")

# Nonparametric Mann-Whitney U test (Wilcoxon rank-sum)

u_stat, u_p_val = mannwhitneyu(group_pos, group_neg, alternative='two-sided')
print(f"Mann-Whitney U: U-statistic = {u_stat:.4f}, p-value = {u_p_val:.4g}")

# Wilcoxon signed-rank test (only if paired and same length, here for completeness)

if len(group_pos) == len(group_neg):
    try:
        w_stat, w_p_val = wilcoxon(group_pos.values, group_neg.values)
        print(f"Wilcoxon signed-rank: W-statistic = {w_stat:.4f}, p-value = {w_p_val:.4g}")
    except ValueError as e:
        print(f"Wilcoxon test not applicable: {e}")
else:
    print("Wilcoxon signed-rank test not run (groups have different lengths).")

# Correlate Variable with All Other Variables

In [ ]:
x_col = 'flipped_lesions_all_datasets_weighted_avg_r_map_cosine'

In [ ]:
from calvin_utils.statistical_utils.correlation_barplot import CorrelationBarPlot
plotter = CorrelationBarPlot(data_df, x_col=x_col, method="spearman")
corr_series = plotter.run(save_path=None)

Focus on correlations above a threhsold

In [ ]:
strong_corrs = plotter.plot_threshold(0.3)      # |r| ≥ 0.4 plot